<a href="https://colab.research.google.com/github/SahputraS/Outbreak-Simulation-and-Detection-Testing/blob/main/Outbreak_Detection_Evaluation_(Mosquito_Borne_Disease).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The outbreak detection evaluation framework in this analysis is based on dengue data from the dataset published in:
https://journals.plos.org/plosntds/article?id=10.1371/journal.pntd.0009259

In [125]:
import numpy as np
import pandas as pd
import string
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import  f1_score, precision_score, recall_score, accuracy_score

## Open Dataset

In [126]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [127]:
dengue_cucuta_path = '/content/drive/My Drive/Mosquito_Data/dengue_cucuta.csv'

dengue_cucuta = pd.read_csv(dengue_cucuta_path)
dengue_cucuta.head(3)

,Municipality,Year,Week_inY,Reported_Cases,Week,Farrington_Flexible,EARS,ARIMA
0,Cucuta,2009,1,52,105,0,0,0
1,Cucuta,2009,2,44,106,0,0,0
2,Cucuta,2009,3,49,107,0,0,0


In [128]:
dengue_bemon_path = '/content/drive/My Drive/Mosquito_Data/dengue_bemon.csv'

dengue_bemon = pd.read_csv(dengue_bemon_path)
dengue_bemon.head(3)

,Municipality,Year,Week_inY,Reported_Cases,Week,Farrington_Flexible,EARS,ARIMA
0,Bello+Moniquira,2009,1,2,105,0,0,0
1,Bello+Moniquira,2009,2,1,106,0,0,0
2,Bello+Moniquira,2009,3,1,107,0,0,0


## Label The Ground Truth
Labeling the outbreak on the datapoints that are above the 75th percentile. This is based on the approach of https://www.nature.com/articles/s41598-024-81367-1.

In [129]:
cases_cucuta = dengue_cucuta.drop(columns=['Year', 'Week_inY', 'Farrington_Flexible', 'EARS', 'ARIMA'])

cc = cases_cucuta['Reported_Cases']
thres_percent = 75
thres_cucuta = np.percentile(cc, thres_percent)

cases_cucuta['data_label'] = np.where(cases_cucuta['Reported_Cases'] > thres_cucuta, 'dengue', 'endemic')
cases_cucuta.head(3)

,Municipality,Reported_Cases,Week,data_label
0,Cucuta,52,105,endemic
1,Cucuta,44,106,endemic
2,Cucuta,49,107,endemic


In [130]:
cases_bemon = dengue_bemon.drop(columns=['Year', 'Week_inY', 'Farrington_Flexible', 'EARS', 'ARIMA'])

bm = cases_bemon['Reported_Cases']
thres_bemon = np.percentile(bm, thres_percent)
print('The threshold for dengue in Bello + Moniquira Mun. is', thres_bemon, 'cases')

cases_bemon['data_label'] = np.where(cases_bemon['Reported_Cases'] > thres_bemon, 'dengue', 'endemic')
cases_bemon.head(3)

The threshold for dengue in Bello + Moniquira Mun. is 17.0 cases


,Municipality,Reported_Cases,Week,data_label
0,Bello+Moniquira,2,105,endemic
1,Bello+Moniquira,1,106,endemic
2,Bello+Moniquira,1,107,endemic


## Formatting The DataFrame to Work With Epi-Quark

In [131]:
def complete_data(df, x1, x2, label_col, val_col, fill_val=0, extra_labels=None):
    """
    Expands the DataFrame to all combinations of x1, x2, label_col,
    filling missing value_col with fill_value (default=0).

    INPUT
    df : Input DataFrame
    x1 : Column name of the first axis
    x2 : Column name of the second axis
    label_col : Name of the label column.
    val_col : Name of the value column to fill.
    fill_val: Value to fill for missing entries (default: 0).
    """
    # Take unique values from each axis and disease label
    all_x1 = df[x1].unique()
    all_x2 = df[x2].unique()
    all_labels = df[label_col].unique()

    if extra_labels is not None: # Added the endemic and non_case if not already exist in the data
      all_labels = np.unique(np.concatenate([all_labels, np.array(extra_labels)]))

    # Build full index
    full_index = pd.MultiIndex.from_product([all_x1, all_x2, all_labels], names=[x1, x2, label_col])
    full_grid = full_index.to_frame(index=False)


    merged = pd.merge(
        full_grid,
        df[[x1, x2, label_col, val_col]],
        on=[x1, x2, label_col],
        how='left'
    )

    # Fill the missing values with 0
    original_dtype = df[val_col].dtype
    merged[val_col] = merged[val_col].fillna(fill_val).astype(original_dtype)

    merged = merged.drop_duplicates()

    return merged

### Formatting the cases dataframe

In [132]:
cases = pd.concat([cases_cucuta, cases_bemon], ignore_index=True)
cases = cases.rename(columns={'Reported_Cases': 'value'})
cases.head(4)

,Municipality,value,Week,data_label
0,Cucuta,52,105,endemic
1,Cucuta,44,106,endemic
2,Cucuta,49,107,endemic
3,Cucuta,45,108,endemic


In [133]:
cases = complete_data(cases, x1='Municipality', x2='Week', label_col='data_label', val_col='value')
cases.head(4)

,Municipality,Week,data_label,value
0,Cucuta,105,endemic,52
1,Cucuta,105,dengue,0
2,Cucuta,106,endemic,44
3,Cucuta,106,dengue,0


### Formatting the signal dataframe

In [134]:
signal_all = pd.concat([dengue_cucuta, dengue_bemon], ignore_index=True).drop(columns=['Year', 'Week_inY'])

In [135]:
def signal_split_label(df, col):
    default_cols = ['Municipality', 'Week', 'Reported_Cases']
    all_cols = default_cols + [col]
    sig = df[all_cols].copy()

    # Label the signal
    sig['signal_label'] = 'non_case'
    sig.loc[(sig[col] == 1), 'signal_label'] = 'outbreak_dengue'
    sig.loc[(sig[col] == 0) & (sig['Reported_Cases'] > 0), 'signal_label'] = 'endemic'
    return sig

In [144]:
signal_FF = signal_split_label(signal_all, 'Farrington_Flexible')
signal_FF['value'] = 0.75
signal_FF = complete_data(signal_FF, x1='Municipality', x2='Week', label_col='signal_label', val_col='value')
signal_FF.head(3)

,Municipality,Week,signal_label,value
0,Cucuta,105,endemic,0.75
1,Cucuta,105,outbreak_dengue,0.00
2,Cucuta,105,non_case,0.00


## Epi-Quark

In [138]:
try:
    from epiquark import conf_matrix, score, timeliness
except ImportError:
    import sys
    !{sys.executable} -m pip install git+https://github.com/aauss/epi-quark.git
    from epiquark import conf_matrix, score, timeliness

  available scoring metrics:       
-   "f1": sk_metrics.f1_score,
-   "brier": sk_metrics.brier_score_loss,
-   "auc": _auc,
-  "sensitivity": _sensitivity,
-  "recall": _sensitivity,
-  "tpr": _sensitivity,
-  "specificity": _specificity,
-  "tnr": _specificity,
-  "fpr": _fpr,
-  "fnr": _fnr,
-  "precision": _precision,
-  "ppv": _precision,
-  "npv": _npv,
-  "matthews": sk_metrics.matthews_corrcoef,
-  "r2": sk_metrics.r2_score,
-  "mse": sk_metrics.mean_squared_error,
-  "mae": sk_metrics.mean_absolute_error,

### Farrington Flexible Evaluation

In [143]:
metrics_epi_quark_ff = {
    "precision": score(cases, signal_FF, "precision", 0.49, 0.49),
    "recall":    score(cases, signal_FF, "recall", 0.49, 0.49),
    "f1":        score(cases, signal_FF, "f1", 0.49, 0.49),
}

epi_quark_ff = pd.DataFrame(metrics_epi_quark_ff, index=['dengue', 'endemic', 'non_case'])
epi_quark_ff = epi_quark_ff.round(2)
display(epi_quark_ff)

/usr/local/lib/python3.11/dist-packages/epiquark/scorer.py:177: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({"p(d,s|x)": sum})
/usr/local/lib/python3.11/dist-packages/epiquark/scorer.py:177: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({"p(d,s|x)": sum})
/usr/local/lib/python3.11/dist-packages/epiquark/scorer.py:177: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({"p(d,s|x)": sum})


,precision,recall,f1
dengue,0.33,0.13,0.18
endemic,0.76,0.91,0.83
non_case,1.00,1.00,1.00
